In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import bs4 as bs
import urllib.request as url

In [3]:
extract_url = url.urlopen('https://en.wikipedia.org/wiki/University_of_Calgary#:~:text=The%20University%20of%20Calgary%20(U,separate%2C%20autonomous%20university%20in%201966.')
raw_html = extract_url.read()

article = bs.BeautifulSoup(raw_html, 'lxml')
article = article.find_all('p')
print(article[:3])

[<p>The <b>University of Calgary</b> (<b>U of C</b> or <b>UCalgary</b>) is a <a href="/wiki/Public_university" title="Public university">public</a> <a href="/wiki/Research_university" title="Research university">research university</a> located in <a href="/wiki/Calgary" title="Calgary">Calgary</a>, <a href="/wiki/Alberta" title="Alberta">Alberta</a>, Canada. The University of Calgary started in 1944 as the Calgary branch of the <a href="/wiki/University_of_Alberta" title="University of Alberta">University of Alberta</a>, founded in 1908, prior to being instituted into a separate, autonomous university in 1966. It is composed of 14 faculties and over 85 research institutes and centres. The main campus is located in the northwest quadrant of the city near the Bow River and a smaller south campus is located in the city centre. The main campus houses most of the research facilities and works with provincial and federal research and regulatory agencies, several of which are housed next to t

In [4]:
art_text = ''
for para in article:
    art_text += para.text
print(art_text[:1000])

The University of Calgary (U of C or UCalgary) is a public research university located in Calgary, Alberta, Canada. The University of Calgary started in 1944 as the Calgary branch of the University of Alberta, founded in 1908, prior to being instituted into a separate, autonomous university in 1966. It is composed of 14 faculties and over 85 research institutes and centres. The main campus is located in the northwest quadrant of the city near the Bow River and a smaller south campus is located in the city centre. The main campus houses most of the research facilities and works with provincial and federal research and regulatory agencies, several of which are housed next to the campus such as the Geological Survey of Canada. The main campus covers approximately 200 hectares (490 acres).
A member of the U15, the University of Calgary is also one of Canada's top research universities (based on the number of Canada Research Chairs). The university has a sponsored research revenue of $380.4

In [5]:
import re
from nltk.stem import WordNetLemmatizer

art_text = re.sub(r'\[[0-9]*\]', ' ', art_text)
art_text = re.sub(r'\s+', ' ', art_text)
lemmatizer = WordNetLemmatizer()
sent_tokens = nltk.sent_tokenize(art_text)
sent_tokens[:1]
word_tokens = nltk.word_tokenize(art_text)
word_tokens[:20]

['The',
 'University',
 'of',
 'Calgary',
 '(',
 'U',
 'of',
 'C',
 'or',
 'UCalgary',
 ')',
 'is',
 'a',
 'public',
 'research',
 'university',
 'located',
 'in',
 'Calgary',
 ',']

In [6]:
def lemmatization(docs):
    
    lemmat = [lemmatizer.lemmatize(w.lower()) for w in docs if w not in set(stopwords.words('English'))]
    return lemmat
def pre_processing(doc):
    doc = doc.lower()
    ignore_words = ['!', ',', '$', '&', '@', '.','?']
    words = lemmatization(doc)
    words = [w for w in words if w not in ignore_words]
    return words
print(pre_processing(art_text[:10]))
        

['h', 'e', ' ', 'u', 'n', 'v', 'e', 'r']


In [7]:
import random

greeting_inputs = ("Hey", "Good morning", "Good evening", "morning", "evening", "Hi", "Wassup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello"]

def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)
  

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input):
    uofc_response = ''
    sent_tokens.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=pre_processing, stop_words='english')
    all_word_vectors = word_vectorizer.fit_transform(sent_tokens)
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        uofc_response = uofc_response + "I am sorry, I could not understand you"
        return uofc_response
    else:
        uofc_response = uofc_response + sent_tokens[similar_sentence_number]
        return uofc_response

In [9]:
word_vectorizer = TfidfVectorizer(tokenizer=pre_processing, stop_words='english')
all_word_vectors = word_vectorizer.fit_transform(word_tokens)

similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

similar_sentence_number = similar_vector_values.argsort()[0][-2]

C:\Users\Vivek\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'e', 'f', 'g', 'h', 'k', 'l', 'n', 'p', 'r', 'u', 'v', 'w', 'x'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [10]:
import pyttsx3
import speech_recognition as sr
import wikipedia
from newspaper import Article
import webbrowser
import datetime
def talk(string):
    engine = pyttsx3.init('sapi5')
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    engine.say(string)
    engine.runAndWait()
def questi():
    r=sr.Recognizer()
    
    with sr.Microphone() as source:
        read = r.listen(source)
        text = r.recognize_google(read)
        return text
def wiki(string):
    return(wikipedia.summary(string, sentences = 2))


In [25]:
hello_dino = "hey dino"
continue_dialogue = True

while(continue_dialogue == True):
    human_text = questi().lower()
    human_text = human_text.lower()
    print(human_text)
    if (human_text == hello_dino):
        talk("Hello, How can I help you today")
        print("Hello, How can I help you today ")
    if 'Who are you' in human_text:
        response = 'I am Dino, your assistant, built by Vivek from University of Calgary. I am here to help you with what you are looking for. However, I am not completely built, but I could assist you with what I am programmed for'
        print(response)
        talk(response)
    elif 'open youtube' in human_text:
        talk('OPening Youtube')
        result = webbrowser.open('www.youtube.com')
    elif 'open google' in human_text:
        talk('Opening google')
        result = webbrowser.open('www.google.com')
    elif 'what is the time' in human_text:
        currenttime = datetime.datetime.now().strftime("%H:%M:%S")    
        talk(f'the time is {currenttime}')
    
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            continue_dialogue = False
            talk("You're most welcome")
            print("dino: Most welcome")
        
        else:
            if generate_greeting_response(human_text) != None:
                talk("dino: " + generate_greeting_response(human_text))
                print("dino: " + generate_greeting_response(human_text))
            else:
                print("Dino: ", end="")
                resp = generate_response(human_text)
                talk(resp)
                print(resp)
                sent_tokens.remove(human_text)
                
                
    else:
        continue_dialogue = False
        talk("Good bye and take care of yourself")
        print("Dino: Good bye and take care of yourself...")

hey dino
Hello, How can I help you today 
Dino: This site became known as the Health Sciences Campus, with subsequent additions being the Heritage Medical Building (opened 1987), the Health Research Innovation Centre and the Teaching, Research and Wellness building.
who are you
Dino: It is published every weekday throughout the year.
tell me about the fitness centre at the university of calgary
Dino: Members of the board of governors include the chancellor of the university, the university president, two alumni nominated by the alumni association, a member of the university senate, nine members of the general public, as well as three students, and three staff members nominated by their respective associations.
in which year was the university of calgary founded
Dino: The main campus houses most of the research facilities and works with provincial and federal research and regulatory agencies, several of which are housed next to the campus such as the Geological Survey of Canada.
bye
Din